In [ ]:
!nvidia-smi

Thu Feb 16 10:38:18 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   57C    P0    28W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install datasets
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 KB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 3.2 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 45.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 82.9 MB/s eta 0:00:00


In [ ]:
from pathlib import Path
from datasets import load_dataset
dataset = load_dataset("csv", data_files="/content/train.csv")["train"].train_test_split(test_size=0.1)
dataset

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-fd82f6759bf1b2af/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['TITLE', 'ABSTRACT', 'Computer Science', 'Physics', 'Mathematics', 'Statistics', 'Quantitative Biology', 'Quantitative Finance'],
        num_rows: 18874
    })
    test: Dataset({
        features: ['TITLE', 'ABSTRACT', 'Computer Science', 'Physics', 'Mathematics', 'Statistics', 'Quantitative Biology', 'Quantitative Finance'],
        num_rows: 2098
    })
})

In [ ]:
dataset["train"][10]

{'TITLE': 'T-duality in rational homotopy theory via $L_\\infty$-algebras',
 'ABSTRACT': "  We combine Sullivan models from rational homotopy theory with Stasheff's\n$L_\\infty$-algebras to describe a duality in string theory. Namely, what in\nstring theory is known as topological T-duality between $K^0$-cocycles in type\nIIA string theory and $K^1$-cocycles in type IIB string theory, or as Hori's\nformula, can be recognized as a Fourier-Mukai transform between twisted\ncohomologies when looked through the lenses of rational homotopy theory. We\nshow this as an example of topological T-duality in rational homotopy theory,\nwhich in turn can be completely formulated in terms of morphisms of\n$L_\\infty$-algebras.\n",
 'Computer Science': 0,
 'Physics': 0,
 'Mathematics': 1,
 'Statistics': 0,
 'Quantitative Biology': 0,
 'Quantitative Finance': 0}

In [ ]:
cols = dataset["train"].column_names
dataset = dataset.map(lambda x : {"labels": [x[c] for c in cols if c != "TITLE" and c!="ABSTRACT"]})
dataset

  0%|          | 0/18874 [00:00<?, ?ex/s]

  0%|          | 0/2098 [00:00<?, ?ex/s]

DatasetDict({
    train: Dataset({
        features: ['TITLE', 'ABSTRACT', 'Computer Science', 'Physics', 'Mathematics', 'Statistics', 'Quantitative Biology', 'Quantitative Finance', 'labels'],
        num_rows: 18874
    })
    test: Dataset({
        features: ['TITLE', 'ABSTRACT', 'Computer Science', 'Physics', 'Mathematics', 'Statistics', 'Quantitative Biology', 'Quantitative Finance', 'labels'],
        num_rows: 2098
    })
})

In [ ]:
dataset["train"][1000]

{'TITLE': 'Coarse-grained simulation of auxetic, two-dimensional crystal dynamics',
 'ABSTRACT': '  The increasing number of protein-based metamaterials demands reliable and\nefficient methods to study the physicochemical properties they may display. In\nthis regard, we develop a simulation strategy based on Molecular Dynamics (MD)\nthat addresses the geometric degrees of freedom of an auxetic two-dimensional\nprotein crystal. This model consists of a network of impenetrable rigid squares\nlinked through massless rigid rods, thus featuring a large number of both\nholonomic and nonholonomic constraints. Our MD methodology is optimized to\nstudy highly constrained systems and allows for the simulation of long-time\ndynamics with reasonably large timesteps. The data extracted from the\nsimulations shows a persistent motional interdependence among the protein\nsubunits in the crystal. We characterize the dynamical correlations featured by\nthese subunits and identify two regimes characteri

In [ ]:
from transformers import AutoTokenizer, TFDistilBertModel
import tensorflow as tf

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased",problem_type="multi_label_classification")


In [ ]:
def tokenize_and_encode(examples):
  return tokenizer(examples["TITLE"],examples["ABSTRACT"], truncation=True)

In [ ]:
cols = dataset["train"].column_names
cols.remove("labels")
dataset_enc = dataset.map(tokenize_and_encode, batched=True, remove_columns=cols)
dataset_enc

  0%|          | 0/19 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 18874
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 2098
    })
})

In [ ]:
import torch

In [ ]:
dataset_enc.set_format("torch")
dataset_enc = (dataset_enc
          .map(lambda x : {"float_labels": x["labels"].to(torch.float)}, remove_columns=["labels"])
          .rename_column("float_labels", "labels"))

  0%|          | 0/18874 [00:00<?, ?ex/s]

  0%|          | 0/2098 [00:00<?, ?ex/s]

In [ ]:
from transformers import (AutoModelForSequenceClassification, 
                          TrainingArguments, Trainer)

In [ ]:
num_labels=6
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=num_labels, problem_type="multi_label_classification").to('cuda')

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias', 'pre_classifier

In [ ]:
args = TrainingArguments(".", num_train_epochs=10)

trainer = Trainer(model=model, args=args, train_dataset=dataset_enc["train"], eval_dataset=dataset_enc["test"], tokenizer=tokenizer)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 2098
  Batch size = 8


{'eval_loss': 0.26193374395370483,
 'eval_runtime': 26.8825,
 'eval_samples_per_second': 78.043,
 'eval_steps_per_second': 9.783}

In [ ]:
trainer.train()

***** Running training *****
  Num examples = 18874
  Num Epochs = 10
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 23600
  Number of trainable parameters = 66958086


Step,Training Loss
500,0.216700
1000,0.207100
1500,0.189700
2000,0.179800
2500,0.176500
3000,0.142300
3500,0.147600
4000,0.151600
4500,0.143100
5000,0.121900


Saving model checkpoint to ./checkpoint-500
Configuration saved in ./checkpoint-500/config.json
Model weights saved in ./checkpoint-500/pytorch_model.bin
tokenizer config file saved in ./checkpoint-500/tokenizer_config.json
Special tokens file saved in ./checkpoint-500/special_tokens_map.json
Saving model checkpoint to ./checkpoint-1000
Configuration saved in ./checkpoint-1000/config.json
Model weights saved in ./checkpoint-1000/pytorch_model.bin
tokenizer config file saved in ./checkpoint-1000/tokenizer_config.json
Special tokens file saved in ./checkpoint-1000/special_tokens_map.json
Saving model checkpoint to ./checkpoint-1500
Configuration saved in ./checkpoint-1500/config.json
Model weights saved in ./checkpoint-1500/pytorch_model.bin
tokenizer config file saved in ./checkpoint-1500/tokenizer_config.json
Special tokens file saved in ./checkpoint-1500/special_tokens_map.json
Saving model checkpoint to ./checkpoint-2000
Configuration saved in ./checkpoint-2000/config.json
Model weig

Step,Training Loss
500,0.216700
1000,0.207100
1500,0.189700
2000,0.179800
2500,0.176500
3000,0.142300
3500,0.147600
4000,0.151600
4500,0.143100
5000,0.121900


Saving model checkpoint to ./checkpoint-17000
Configuration saved in ./checkpoint-17000/config.json
Model weights saved in ./checkpoint-17000/pytorch_model.bin
tokenizer config file saved in ./checkpoint-17000/tokenizer_config.json
Special tokens file saved in ./checkpoint-17000/special_tokens_map.json
Saving model checkpoint to ./checkpoint-17500
Configuration saved in ./checkpoint-17500/config.json
Model weights saved in ./checkpoint-17500/pytorch_model.bin
tokenizer config file saved in ./checkpoint-17500/tokenizer_config.json
Special tokens file saved in ./checkpoint-17500/special_tokens_map.json
Saving model checkpoint to ./checkpoint-18000
Configuration saved in ./checkpoint-18000/config.json
Model weights saved in ./checkpoint-18000/pytorch_model.bin
tokenizer config file saved in ./checkpoint-18000/tokenizer_config.json
Special tokens file saved in ./checkpoint-18000/special_tokens_map.json
Saving model checkpoint to ./checkpoint-18500
Configuration saved in ./checkpoint-18500/

TrainOutput(global_step=23600, training_loss=0.06539904214062933, metrics={'train_runtime': 6932.5232, 'train_samples_per_second': 27.225, 'train_steps_per_second': 3.404, 'total_flos': 1.7914917655601136e+16, 'train_loss': 0.06539904214062933, 'epoch': 10.0})

In [ ]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 2098
  Batch size = 8


{'eval_loss': 0.5321647524833679,
 'eval_runtime': 23.3913,
 'eval_samples_per_second': 89.691,
 'eval_steps_per_second': 11.243,
 'epoch': 10.0}

In [ ]:
s=""

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
!unzip /content/gdrive/MyDrive/checkpoint-23500.zip -d /content/checkpoint-23500

Archive:  /content/gdrive/MyDrive/checkpoint-23500.zip
  inflating: /content/checkpoint-23500/checkpoint-23500/config.json  
  inflating: /content/checkpoint-23500/checkpoint-23500/optimizer.pt  
  inflating: /content/checkpoint-23500/checkpoint-23500/pytorch_model.bin  
  inflating: /content/checkpoint-23500/checkpoint-23500/rng_state.pth  
  inflating: /content/checkpoint-23500/checkpoint-23500/scheduler.pt  
  inflating: /content/checkpoint-23500/checkpoint-23500/special_tokens_map.json  
  inflating: /content/checkpoint-23500/checkpoint-23500/tokenizer.json  
  inflating: /content/checkpoint-23500/checkpoint-23500/tokenizer_config.json  
  inflating: /content/checkpoint-23500/checkpoint-23500/trainer_state.json  
  inflating: /content/checkpoint-23500/checkpoint-23500/training_args.bin  
  inflating: /content/checkpoint-23500/checkpoint-23500/vocab.txt  


In [ ]:
import torch
model=torch.load("/content/checkpoint-23500/checkpoint-23500")

IsADirectoryError: ignored

In [ ]:
from transformers import AutoModelForSequenceClassification
import torch

In [ ]:
num_labels=6
model = AutoModelForSequenceClassification.from_pretrained("/content/checkpoint-23500/checkpoint-23500", num_labels=num_labels, problem_type="multi_label_classification")

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("/content/checkpoint-23500/checkpoint-23500",problem_type="multi_label_classification")

In [ ]:
def tokenize_and_encode(examples):
  return tokenizer(examples["TITLE"],examples["ABSTRACT"], truncation=True)

In [ ]:
from pathlib import Path
from datasets import load_dataset
dataset = load_dataset("csv", data_files="/content/test.csv")
dataset

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-587222b24855640c/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['TITLE', 'ABSTRACT'],
        num_rows: 8989
    })
})

In [ ]:
cols = dataset["train"].column_names
dataset_enc = dataset.map(tokenize_and_encode, batched=True, remove_columns=cols)

  0%|          | 0/9 [00:00<?, ?ba/s]

In [ ]:
dataset_enc

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 8989
    })
})

In [ ]:
from transformers import pipeline
classifier = pipeline('text-classification', model=model, tokenizer=tokenizer)

In [ ]:
import pandas as pd
df=pd.read_csv("test.csv")

In [ ]:
classifier([str(df.iloc[0,0]),str(df.iloc[0,1])])

[{'label': 'LABEL_3', 'score': 0.9999880790710449},
 {'label': 'LABEL_3', 'score': 0.9999948740005493}]

In [ ]:
inputs = tokenizer(str(df.iloc[0,0]),str(df.iloc[0,1]), return_tensors="pt")

In [ ]:
op=model(**inputs).logits

In [ ]:
m=torch.nn.Softmax(dim=1)
#m=input = torch.randn(2, 3)
output = m(op)

In [ ]:
output>0.5

tensor([[False, False, False,  True, False, False]])

In [ ]:
#stat
str(df.iloc[0,0])

'Closed-form Marginal Likelihood in Gamma-Poisson Matrix Factorization'